# Video Classification using CNN-LSTM Model

This Jupyter notebook contains code for a video classification model using a combination of Convolutional Neural Networks (CNN) and Long Short-Term Memory (LSTM) networks. The model is trained on a custom dataset consisting of four classes: 'WalkingWithDog', 'TaiChi', 'Swing', and 'HorseRace'.

## Table of Contents

1. [Imports and Setup](#imports-and-setup)
2. [Data Preparation](#data-preparation)
3. [Model Architecture](#model-architecture)
4. [Training the Model](#training-the-model)
5. [Evaluation and Saving the Model](#evaluation-and-saving-the-model)

## Imports and Setup

The code begins by importing the necessary libraries and modules. It also sets up the environment and defines some constants.

```python
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from keras.layers import *
from keras.models import Sequential
from keras.utils import to_categorical
from tqdm import tqdm

from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.regularizers import l2

os.system('cls')

seed_constant=27
np.random.seed()

image_height, image_width= 64,64
sequence_length= 20
data_dir='UCF50'

classes_list=['WalkingWithDog' ,'TaiChi','Swing','HorseRace']
```

## Data Preparation

The `frames_extraction` function is defined to extract frames from a video file. The `create_dataset` function then uses this function to create a dataset of frames and corresponding labels. The dataset is split into training and testing sets using the `train_test_split` function from scikit-learn.

```python
def frames_extraction(video_path):
    # ...

def create_dataset():
    # ...

features,labels= create_dataset()
one_hot_encoded_label=to_categorical(labels)

train_features, test_features, train_labels, test_labels = train_test_split(
    features, one_hot_encoded_label, test_size=0.25, random_state=seed_constant)
```

## Model Architecture

The `create_LRCN_model` function defines the architecture of the CNN-LSTM model. The model consists of two TimeDistributed Conv2D layers, each followed by BatchNormalization, MaxPool2D, and Dropout layers. The output of these layers is then flattened and fed into an LSTM layer, followed by a Dense layer with softmax activation for classification.

```python
def create_LRCN_model():
    # ...

model = create_LRCN_model()
```

## Training the Model

The model is compiled with the Adam optimizer and categorical cross-entropy loss. Early stopping and learning rate reduction are applied to prevent overfitting. The model is then trained on the training set for 100 epochs with a batch size of 20.

```python
opt=Adam(learning_rate=0.0001)
model.compile( optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=0.00001)

model_history = model.fit(train_features, train_labels, epochs=100, batch_size=20,
                          shuffle=True, validation_split=0.2, callbacks=[ lr_reduction ,early_stopping])
```

## Evaluation and Saving the Model

The trained model is evaluated on the testing set, and its accuracy and loss are printed. The model is then saved to a file named 'cnn_lstm_classification_me.h5'.

```python
model_test_eval = model.evaluate(test_features, test_labels)

model.save('cnn_lstm_classification_me.h5')
```

Additionally, the training history is plotted to visualize the training and validation accuracy and loss over epochs.

```python
plt.figure(figsize=(12, 6))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(model_history.history['accuracy'] )
plt.plot(model_history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()
```

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import cv2
import random 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from keras.layers import *
from keras.models import Sequential
from keras.utils import to_categorical
from tqdm import tqdm

from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau 
from keras.optimizers import Adam
from keras.regularizers import l2

os.system('cls')

seed_constant=27
np.random.seed()



image_height, image_width= 64,64
sequence_length= 20
data_dir='UCF50'


classes_list=['WalkingWithDog' ,'TaiChi','Swing','HorseRace']


def frames_extraction(video_path):
  
    frames_list=[]
    video_reader= cv2.VideoCapture(video_path)
    for _ in range(sequence_length):
        success,frame= video_reader.read()
        if not success: break

        resized_frame=cv2.resize(frame,(image_height,image_width))
        normalized_frame= resized_frame/255
        frames_list.append(normalized_frame)

    video_reader.release()    
    return frames_list

def create_dataset():
    features=[]
    labels=[]
    for class_index , class_name in enumerate(classes_list):
        print(f'Extracting data of class : {class_name}')
        file_list=os.listdir(os.path.join(data_dir,class_name))
        for file in tqdm(file_list):
            video_file_path=os.path.join(data_dir,class_name,file)
            frames=frames_extraction(video_file_path)
            if len(frames) == sequence_length:
                features.append(frames)
                labels.append(class_index)


    features=np.array(features)
    labels=np.array(labels)    
    return features,labels


features,labels= create_dataset()
one_hot_encoded_label=to_categorical(labels)
        


 # Split the dataset into train and test sets
train_features, test_features, train_labels, test_labels = train_test_split(
    features, one_hot_encoded_label, test_size=0.25, random_state=seed_constant)

print("Train features shape:", train_features.shape)
print("Train labels shape:", train_labels.shape)
print("Test features shape:", test_features.shape)
print("Test labels shape:", test_labels.shape)


def create_LRCN_model():
    l2_reg = 0.01  # You can adjust this value based on your preference

    model = Sequential()
    model.add(TimeDistributed(Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu', kernel_regularizer=l2(l2_reg)),
                              input_shape=(sequence_length, image_height, image_width, 3)))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPool2D(pool_size=(4, 4))))
    model.add(TimeDistributed(Dropout(0.5)))

    model.add(TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu', kernel_regularizer=l2(l2_reg))))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPool2D(pool_size=(4, 4))))
    model.add(TimeDistributed(Dropout(0.5)))

    # model.add(TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu', kernel_regularizer=l2(l2_reg))))
    # model.add(TimeDistributed(BatchNormalization()))
    # model.add(TimeDistributed(MaxPool2D(pool_size=(4, 4))))
    # model.add(TimeDistributed(Dropout(0.8)))

    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(32, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    # model.add(LSTM(32),k)
    model.add(Dense(len(classes_list), activation='softmax', kernel_regularizer=l2(l2_reg)))



    model.summary()

    return model


model = create_LRCN_model()

opt=Adam(learning_rate=0.0001)
model.compile( optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

# Apply early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=0.00001)

# Store the training history
model_history = model.fit(train_features, train_labels, epochs=100, batch_size=20,
                          shuffle=True, validation_split=0.2, callbacks=[ lr_reduction ,early_stopping])
# Plot training history
plt.figure(figsize=(12, 6))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(model_history.history['accuracy'] )
plt.plot(model_history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

model_test_eval = model.evaluate(test_features, test_labels)

model.save('cnn_lstm_classification_me.h5')